# Compute mistakenness

### Init fiftyone

In [1]:
import os
from pathlib import Path
config_path = Path.home() / ".fiftyone" / "config.global_mongodb.json"
os.environ["FIFTYONE_CONFIG_PATH"] = str(config_path)

In [ ]:
import fiftyone as fo
fo.list_datasets()

### Launch app

In [ ]:
dataset_name = "TRAIN_RL_SPLIT_THERMAL_2024_03"
dataset = fo.load_dataset(dataset_name)
session = fo.launch_app(dataset=dataset, port=2203, address="127.0.0.1", auto=False)
session.show()

### Cast labels to classes used by prediction model (yolov5)

The class_map.yaml file can be found here: https://github.com/SEA-AI/yolov5/blob/sea.ai/data/class_map.yaml

In [20]:
import yaml 

def load_class_map(file_path):
    with open(file_path, 'r') as file:
        return yaml.safe_load(file)
    
def cast_label(label, cast_dict):
    if label in cast_dict:
        return cast_dict[label]
    return label
    
class_map = load_class_map("class_map.yaml")

print("Test: HUMAN_IN_WATER -->", cast_label("HUMAN_IN_WATER", class_map))


Test: HUMAN_IN_WATER --> HUMAN


In [7]:
dataset = fo.load_dataset(dataset_name)

for sample in dataset.iter_samples(progress=True):
    
    detections_list = []

    if sample.ground_truth_det is not None:

        for det in sample.ground_truth_det.detections:

            label = cast_label(det.label, class_map)
            if label == "None":
                continue
            
            detection = fo.Detection(label=label, bounding_box=det.bounding_box, index=det.index)

            detections_list.append(detection)
    
    det_list = fo.core.labels.Detections(detections=detections_list)
    sample["ground_truth_det_cast"] = det_list
    sample.save()
dataset.save()

   0% ||--------------|    0/1000 [3.5ms elapsed, ? remaining, ? samples/s] 

 100% |███████████████| 1000/1000 [2.5s elapsed, 0s remaining, 414.3 samples/s]      


### Make predictions

In [9]:
import yolov5
import cv2

model = yolov5.load('SEA-AI/yolov5n-IR')

model.conf = 0.1  # NMS confidence threshold
model.iou = 0.1  # NMS IoU threshold
model.agnostic = True  # NMS class-agnostic

dataset_name = "TRAIN_RL_SPLIT_THERMAL_2024_03"
dataset = fo.load_dataset(dataset_name)

for sample in dataset.iter_samples(progress=True):

    if "ADDED_YOLO_PREDICTION" in sample.tags:
        continue

    frame = cv2.imread(sample.filepath)
    h, w, _ = frame.shape
    results = model(frame, size=640)

    detections = []
    for *box, conf, cls in results.xyxy[0]:
        # FiftyOne expects [top-left-x, top-left-y, width, height] normalized to [0, 1]
        x1, y1, x2, y2 = box
        rel_box = [x1 / w, y1 / h, (x2 - x1) / w, (y2 - y1) / h]
        detections.append(
            fo.Detection(
                label=model.names[int(cls)],
                bounding_box=rel_box,
                confidence=conf,
            )
        )

    det_list = fo.core.labels.Detections(detections=detections)
    sample["yolo_prediction"] = det_list
    sample.tags.append("ADDED_YOLO_PREDICTION")
    sample.save()
    
dataset.save()

 100% |███████████████| 1000/1000 [5.0m elapsed, 0s remaining, 3.5 samples/s]      


### Compute mistakenness

In [ ]:
import fiftyone.brain as fob

# Add this field to prevent cast issued when calculating mistakenness (otherwise the mistakeness score will possibly be casted to integers)
dataset.add_sample_field("mistakenness", fo.FloatField)

"""
    To customize the default settings for mistakenness computation, update the values in the mistakenness.py file located in the fiftyone.brain module:
    (fiftyone/brain/internal/core/mistakenness.py lines 34-35)

    _MISSED_CONFIDENCE_THRESHOLD = 0.70  # Threshold for detection confidence to be considered as missed
    _DETECTION_IOU = 0.2  # Threshold for Intersection over Union (IoU) to consider a detection as a match
"""

fob.compute_mistakenness(dataset, pred_field="yolo_prediction", label_field="ground_truth_det_cast")


### Add mistakenness_loc on sample level

In [26]:
def add_mistakenness_loc_metric(dataset, ground_truth_det="ground_truth_det"):
    # Specify field as float to prevent cast issues
    dataset.add_sample_field("mistakenness_loc", fo.FloatField)

    # Loop through samples
    for sample in dataset:
        detections = sample[ground_truth_det]

        # Extract the mistakenness_loc values from detections
        mistakenness_loc_values = [
            det.mistakenness_loc for det in detections.detections if hasattr(det, "mistakenness_loc")
        ]

        # Assign the maximum mistakenness_loc to the sample field, or -1.0 if there are no detections
        sample["mistakenness_loc"] = max(mistakenness_loc_values, default=-1.0)
        sample.save()
    dataset.save()

# Example call
add_mistakenness_loc_metric(dataset, ground_truth_det="ground_truth_det_cast")

### Sort by mistakenness

In [27]:
mistakenness_view = dataset.sort_by("mistakenness", reverse=True)
session.view = mistakenness_view

### Sort by mistakenness_loc

In [ ]:
mistakenness_view = dataset.sort_by("mistakenness_loc", reverse=True)
session.view = mistakenness_view

### Delete mistakenness metrics

In [ ]:
dataset.delete_sample_field("mistakenness_loc")
dataset.delete_brain_run("mistakenness")
dataset.delete_evaluations()